In [0]:
#Get top 2 students per department by marks


from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.window import Window
from pyspark.sql.functions import sum, rank, dense_rank, row_number

spark = SparkSession.builder.appName("window function").getOrCreate()

data = [("alice", "hr", 350), ("bob", "hr", 345), ("ankit", "ee", 356), ("sam","ee",456), ("reem", "cloud", 765),("anupama", "cloud",987), ("ankita", "hr" , 654)]

columns = ["name", "department","marks"]

df = spark.createDataFrame(data,columns)
df.show()
windowSpec = Window.partitionBy("department").orderBy(col("marks").desc())

df_with_rank = df.withColumn("row_number", row_number().over(windowSpec))
df_with_rank.show()
top_2 = df_with_rank.filter(df_with_rank.row_number <= 2)
top_2.show()


#Now doing using only sql operation 

df.createOrReplaceTempView("students")
df1 = spark.sql("""SELECT name, department, marks
                FROM (SELECT *, row_number() OVER (PARTITION BY department ORDER BY marks DESC) AS rn 
                FROM students) where rn <=2
                """)
df1.show()



